In [1]:
!git clone https://github.com/fenglinglwb/MAT.git

Cloning into 'MAT'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 199 (delta 42), reused 39 (delta 39), pack-reused 148
Receiving objects: 100% (199/199), 18.89 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [2]:
!pip install -r MAT/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86

In [3]:
import torchvision

train = torchvision.datasets.Places365(root = "MAT/rgdata/", split = 'val', small = True, download = True)


100%|██████████| 67498496/67498496 [00:02<00:00, 29474325.87it/s]


Extracting MAT/rgdata/filelist_places365-standard.tar to MAT/rgdata/


100%|██████████| 525158400/525158400 [00:13<00:00, 38096603.24it/s]


Extracting MAT/rgdata/val_256.tar to MAT/rgdata/


In [4]:
from pathlib import Path
import os
import matplotlib.pyplot as plt
import cv2

path = Path("MAT/rgdata/val_256/")

# if we will use train data then it would have folders many of diff objects while in validation set there is no such complicaion
# removing the improper images
count = 0
for i in os.listdir(path):
  img = plt.imread(path/i)
  if img.shape[-1] != 3:
    os.remove(path/i)
    count += 1

print(f"Deleted images with incorrect number of dims : {count}")

Deleted images with incorrect number of dims : 79


In [19]:
ln = len(os.listdir(path))
thres = int(ln * 0.7)

# os.mkdir("MAT/train")
# os.mkdir("MAT/test")

for ind,i in enumerate(os.listdir(path)):
  if ind < thres:
    pass
    pth = str(path) + "/" + i
    img = cv2.imread(pth)
    cv2.imwrite(f"MAT/train/{ind}.jpg", img)
  else:
    pth = str(path) + "/" + i
    img = cv2.imread(pth)
    cv2.imwrite(f"MAT/test/{ind}.jpg", img)

In [22]:
!python MAT/train.py \
    --outdir=output_path \
    --gpus=1 \
    --batch=2 \
    --metrics=fid36k5_full \
    --data=MAT/train \
    --data_val=MAT/test \
    --dataloader=datasets.dataset_512.ImageFolderMaskDataset \
    --mirror=True \
    --cond=False \
    # --cfg=places512 \
    --aug=noaug \
    --generator=networks.mat.Generator \
    --discriminator=networks.mat.Discriminator \
    --loss=losses.loss.TwoStageLoss \
    --pr=0.1 \
    --pl=False \
    --truncation=0.5 \
    --style_mix=0.5 \
    --ema=10 \
    --lr=0.001

Streaming output truncated to the last 5000 lines.
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. Falling back to torch.nn.functional.conv2d().')
/content/MAT/torch_utils/ops/conv2d_gradfix.py:55: UserWarning: conv2d_gradfix not supported on PyTorch 2.3.0+cu121. Falling back to torch.nn.functional.conv2d().
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. Falling back to torch.nn.functional.conv2d().')
/content/MAT/torch_utils/ops/conv2d_gradfix.py:55: UserWarning: conv2d_gradfix not supported on PyTorch 2.3.0+cu121. Falling back to torch.nn.functional.conv2d().
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. Falling back to torch.nn.functional.conv2d().')
/content/MAT/torch_utils/ops/conv2d_gradfix.py:55: UserWarning: conv2d_gradfix not supported on PyTorch 2.3.0+cu121. Falling back to torch.nn.functional.conv2d().
  warnings.warn(f'conv2d_gradfix not supported on PyTorch {torch.__version__}. 

In [24]:
ind

36420